## Identifying Heat Stress Periods Based on Weather Data

This approach focuses on identifying heat stress periods using weather data. Various weather variables, such as temperature, humidity, and the Temperature-Humidity Index (THI), are known to influence heat stress in cows. By setting specific thresholds for these variables, we can pinpoint days when the weather conditions are likely to cause heat stress.

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.optimize import curve_fit, OptimizeWarning
from tqdm import tqdm
import warnings

sns.set_theme()
sns.set_context("notebook")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
dtype_dict = {
    'FarmName_Pseudo': 'str',
    'SE_Number': 'str',
    'AnimalNumber': 'Int64',          
    'StartDate': 'str',
    'StartTime': 'str',
    'DateTime': 'str',
    'LactationNumber': 'Int64',       
    'DaysInMilk': 'Int64', 
    'YearSeason': 'str',           
    'TotalYield': 'float',
    'DateTime': 'str',
    'BreedName': 'str',
    'Age': 'Int64',
    'Mother': 'str',
    'Father': 'str',
    'CullDecisionDate': 'str',
    'Temperature': 'float',
    'RelativeHumidity': 'float',      
    'THI_adj': 'float',
    'HW': 'Int64',                    
    'cum_HW': 'Int64',                
    'Temp15Threshold': 'Int64'        
}


# Load the CSV with specified dtypes
data = pd.read_csv('../Data/MergedData/CleanedYieldData.csv', dtype=dtype_dict)

# Convert date and time columns back to datetime and time objects
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data['StartTime'] = pd.to_datetime(data['StartTime'], format='%H:%M:%S', errors='coerce').dt.time
data['StartDate'] = pd.to_datetime(data['StartDate'], errors='coerce')
data['CullDecisionDate'] = pd.to_datetime(data['CullDecisionDate'], errors='coerce')
data['DateTime'] = pd.to_datetime(data['DateTime'], errors='coerce')
data.head()

,FarmName_Pseudo,SE_Number,AnimalNumber,StartDate,StartTime,LactationNumber,DaysInMilk,TotalYield,DateTime,YearSeason,...,Mother,Father,CullDecisionDate,Temperature,RelativeHumidity,THI_adj,HW,cum_HW,Temp15Threshold,Age
0,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,06:25:00,7,191,13.90,2022-01-01 06:25:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
1,a624fb9a,SE-064c0cec-1189,5189,2022-01-01,16:41:00,7,191,16.87,2022-01-01 16:41:00,2022-1,...,NaN,NaN,2022-12-20,-3.025000,0.930917,28.012944,0,0,0,3095
2,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,15:29:00,7,192,20.41,2022-01-02 15:29:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
3,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,03:31:00,7,192,16.28,2022-01-02 03:31:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096
4,a624fb9a,SE-064c0cec-1189,5189,2022-01-02,22:44:00,7,192,11.53,2022-01-02 22:44:00,2022-1,...,NaN,NaN,2022-12-20,-0.279167,0.990542,32.898193,0,0,0,3096


In [7]:
# Calculate the DailyYield for each cow each day
data['DailyYield'] = data.groupby(['SE_Number', 'StartDate'])['TotalYield'].transform('sum')

# Sort the data by AnimalNumber and StartDate
data.sort_values(['AnimalNumber', 'StartDate'], inplace=True)

# Calculate the previous day's total yield for each cow
data['PreviousDailyYield'] = data.groupby('AnimalNumber')['DailyYield'].shift(1)

# Calculate the daily yield change for each cow
data['DailyYieldChange'] = data['DailyYield'] - data['PreviousDailyYield']

# Group and aggregate data
data = data.groupby(['SE_Number', 'FarmName_Pseudo', 'StartDate']).agg({
    'DailyYield': 'first',
    'PreviousDailyYield': 'first',
    'DailyYieldChange': 'first',
    'HW': 'max',
    'Temperature': 'mean',
    'THI_adj': 'mean',
    'DaysInMilk': 'first',
    'YearSeason': 'first',
    'cum_HW': 'max',
    'Temp15Threshold': 'max',
    'Age': 'first',
    'LactationNumber': 'first'
}).reset_index()

# Renaming and formatting
data.rename(columns={
    'Temperature': 'MeanTemperature',
    'THI_adj': 'MeanTHI_adj',
    'StartDate': 'Date'
}, inplace=True)
data['Date'] = pd.to_datetime(data['Date'])

# Display the first few rows of the transformed data
data.head()

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,LactationNumber
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,30.77,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,7
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,7
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,7
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,7
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,7


## Wood's Lactation Curve
$$
Y(t) = at^be^{-ct}
$$
- $Y(t)$ yield at time $t$ post-calving
- $a$ correlates with the initial production level post-calving
- $b$ governs the incline rate of milk production
- $c$ dictates the decline rate after the peak production 


Normalize the dataset using Wood's lactattion curve and set thresholds for some outliers which have unreasonable values.

In [8]:
# Function to plot the DailyYield and ExpectedYield
def plot_daily_vs_expected(data, farm_id):
    farm_data = data[data['FarmName_Pseudo'] == farm_id]
    unique_animals = farm_data['SE_Number'].unique()

    for animal in unique_animals:
        animal_data = farm_data[farm_data['SE_Number'] == animal]
        plt.figure(figsize=(10, 6))
        plt.plot(animal_data['Date'], animal_data['DailyYield'], label='Daily Yield')
        plt.plot(animal_data['Date'], animal_data['ExpectedYield'], label='Expected Yield', linestyle='--')
        plt.title(f'Daily Yield vs Expected Yield for {animal}')
        plt.xlabel('Date')
        plt.ylabel('Yield')
        plt.legend()
        plt.show()

# Define the Wood's Lactation Curve function
def woods_lactation_curve(dim, a, b, c):
    dim = np.array(dim, dtype=float)
    return a * dim**b * np.exp(-c * dim)

# Function to fit the Wood's Lactation Curve to the dataset
def fit_woods_lactation_curve(dataset):
    # Initialize the 'ExpectedYield' column to NaN
    dataset['ExpectedYield'] = np.nan
    
    # Group the dataset by 'SE_Number' and fit the curve for each cow
    for animal_number, group in tqdm(dataset.groupby('SE_Number'), unit=" Cows"):
        # Prepare the data for fitting
        x_data = group['DaysInMilk'].values
        y_data = group['DailyYield'].values
        
        # Ensure there are no NaN or infinite values in the data
        if not np.isfinite(x_data).all() or not np.isfinite(y_data).all():
            print(f"Non-finite values found for cow {animal_number}, skipping.")
            continue
        
        # Ensure there are enough data points to fit the curve
        if len(x_data) < 50 or len(y_data) < 50:
            print(f"Insufficient data points for cow {animal_number}, skipping.")
            continue
        
        # Fit the model
        try:
            # Initial parameter guesses
            initial_guesses = [max(y_data), 0.4, 0.0001]
            # Bounds on the parameters to prevent overflow
            bounds = ([0, 0, 0], [np.inf, 1, 0.1])
            
            with warnings.catch_warnings():
                warnings.filterwarnings('error', category=OptimizeWarning)
                try:
                    popt, pcov = curve_fit(
                        woods_lactation_curve, x_data, y_data,
                        p0=initial_guesses, bounds=bounds, maxfev=10000
                    )
                    
                    # Predict the expected yield using the fitted model
                    group['ExpectedYield'] = woods_lactation_curve(group['DaysInMilk'], *popt)
                    
                    # Normalize the DailyYield
                    group['NormalizedDailyYield'] = group['DailyYield'] / group['ExpectedYield']
                    
                    # Calculate the daily yield change and normalize it
                    group['PreviousDailyYield'] = group['DailyYield'].shift(1)
                    group['DailyYieldChange'] = group['DailyYield'] - group['PreviousDailyYield']
                    group['NormalizedDailyYieldChange'] = group['DailyYieldChange'] / group['ExpectedYield']
                    
                    # Update the dataset with the fitted data
                    dataset.loc[group.index, 'ExpectedYield'] = group['ExpectedYield']
                    dataset.loc[group.index, 'NormalizedDailyYield'] = group['NormalizedDailyYield']
                    dataset.loc[group.index, 'PreviousDailyYield'] = group['PreviousDailyYield']
                    dataset.loc[group.index, 'DailyYieldChange'] = group['DailyYieldChange']
                    dataset.loc[group.index, 'NormalizedDailyYieldChange'] = group['NormalizedDailyYieldChange']
                
                except OptimizeWarning:
                    print(f"OptimizeWarning for cow {animal_number}, skipping.")
            
        except RuntimeError as e:
            print(f"Curve fit failed for cow {animal_number}: {e}")
        except ValueError as e:
            print(f"Value error for cow {animal_number}: {e}")
    
    # Fill any NaN values in the newly created columns with 0
    dataset['ExpectedYield'] = dataset['ExpectedYield'].fillna(0)
    dataset['NormalizedDailyYield'] = dataset['NormalizedDailyYield'].fillna(0)
    dataset['PreviousDailyYield'] = dataset['PreviousDailyYield'].fillna(0)
    dataset['DailyYieldChange'] = dataset['DailyYieldChange'].fillna(0)
    dataset['NormalizedDailyYieldChange'] = dataset['NormalizedDailyYieldChange'].fillna(0)
    
    return dataset

# Apply the curve fitting function to your dataset
data = fit_woods_lactation_curve(data)

# # Plot the DailyYield and ExpectedYield for the problematic farm
# plot_daily_vs_expected(data, '5c06d92d')

# Check if NormalizedDailyYield is centered around 1 for each unique farm
print("Mean of NormalizedDailyYield:", data.groupby('FarmName_Pseudo')['NormalizedDailyYield'].mean())
print("Standard Deviation of NormalizedDailyYield:", data.groupby('FarmName_Pseudo')['NormalizedDailyYield'].std())
data.head()

  3%|▎         | 42/1277 [00:00<00:08, 150.25 Cows/s]

Insufficient data points for cow SE-5c06d92d-2327, skipping.
Insufficient data points for cow SE-5c06d92d-2328, skipping.
Insufficient data points for cow SE-5c06d92d-2502, skipping.
Insufficient data points for cow SE-5c06d92d-2545, skipping.


 12%|█▏        | 151/1277 [00:00<00:05, 192.65 Cows/s]

Insufficient data points for cow SE-5c06d92d-2815, skipping.
Insufficient data points for cow SE-5c06d92d-2824, skipping.
Insufficient data points for cow SE-5c06d92d-2845, skipping.
Insufficient data points for cow SE-5c06d92d-2885, skipping.


 29%|██▉       | 374/1277 [00:01<00:03, 270.20 Cows/s]

...

 94%|█████████▎| 1197/1277 [00:04<00:00, 277.43 Cows/s]

Insufficient data points for cow SE-f454e660-0779, skipping.
Insufficient data points for cow SE-f454e660-0800, skipping.
Insufficient data points for cow SE-f454e660-0829, skipping.


100%|██████████| 1277/1277 [00:05<00:00, 232.18 Cows/s]


Insufficient data points for cow SE-f454e660-665, skipping.
Insufficient data points for cow SE-f454e660-729, skipping.
Mean of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.995988
752efd72    0.993615
a624fb9a    0.996941
f454e660    0.998148
Name: NormalizedDailyYield, dtype: float64
Standard Deviation of NormalizedDailyYield: FarmName_Pseudo
5c06d92d    0.183936
752efd72    0.170016
a624fb9a    0.222119
f454e660    0.252622
Name: NormalizedDailyYield, dtype: float64


,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,0.00,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,7,29.739372,1.034655,0.000000
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,7,29.692059,1.624003,0.587699
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,7,29.644756,1.029862,-0.596733
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,7,29.597463,1.427825,0.396318
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,7,29.550181,1.302530,-0.127580


In [9]:
# Define heat stress conditions
def identify_weather_based_heat_stress(row):
    if (row['HW'] == 1 or 
        row['cum_HW'] > 0 or 
        row['MeanTHI_adj'] > 61 or
        row['MeanTemperature'] > 15):
        return 1
    return 0

# Apply the function to identify heat stress periods
data['HeatStress'] = data.apply(identify_weather_based_heat_stress, axis=1)

data.head()

,SE_Number,FarmName_Pseudo,Date,DailyYield,PreviousDailyYield,DailyYieldChange,HW,MeanTemperature,MeanTHI_adj,DaysInMilk,YearSeason,cum_HW,Temp15Threshold,Age,LactationNumber,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatStress
0,SE-064c0cec-1189,a624fb9a,2022-01-01,30.77,0.00,0.00,0,-3.025000,28.012944,191,2022-1,0,0,3095,7,29.739372,1.034655,0.000000,0
1,SE-064c0cec-1189,a624fb9a,2022-01-02,48.22,30.77,17.45,0,-0.279167,32.898193,192,2022-1,0,0,3096,7,29.692059,1.624003,0.587699,0
2,SE-064c0cec-1189,a624fb9a,2022-01-03,30.53,48.22,-17.69,0,2.033333,36.760487,193,2022-1,0,0,3097,7,29.644756,1.029862,-0.596733,0
3,SE-064c0cec-1189,a624fb9a,2022-01-04,42.26,30.53,11.73,0,0.066667,31.939524,194,2022-1,0,0,3098,7,29.597463,1.427825,0.396318,0
4,SE-064c0cec-1189,a624fb9a,2022-01-05,38.49,42.26,-3.77,0,-3.700000,26.498206,195,2022-1,0,0,3099,7,29.550181,1.302530,-0.127580,0


In [10]:
# Reorder columns
new_order = [
    "Date", "FarmName_Pseudo", "SE_Number", "Age", "LactationNumber", "DaysInMilk",'YearSeason', "DailyYield", "PreviousDailyYield", 
    "DailyYieldChange", "ExpectedYield", "NormalizedDailyYield", 
    "NormalizedDailyYieldChange", "HeatStress", "Temp15Threshold", "HW", 
    "cum_HW", "MeanTemperature", "MeanTHI_adj"
]
data = data[new_order]
data.head()

,Date,FarmName_Pseudo,SE_Number,Age,LactationNumber,DaysInMilk,YearSeason,DailyYield,PreviousDailyYield,DailyYieldChange,ExpectedYield,NormalizedDailyYield,NormalizedDailyYieldChange,HeatStress,Temp15Threshold,HW,cum_HW,MeanTemperature,MeanTHI_adj
0,2022-01-01,a624fb9a,SE-064c0cec-1189,3095,7,191,2022-1,30.77,0.00,0.00,29.739372,1.034655,0.000000,0,0,0,0,-3.025000,28.012944
1,2022-01-02,a624fb9a,SE-064c0cec-1189,3096,7,192,2022-1,48.22,30.77,17.45,29.692059,1.624003,0.587699,0,0,0,0,-0.279167,32.898193
2,2022-01-03,a624fb9a,SE-064c0cec-1189,3097,7,193,2022-1,30.53,48.22,-17.69,29.644756,1.029862,-0.596733,0,0,0,0,2.033333,36.760487
3,2022-01-04,a624fb9a,SE-064c0cec-1189,3098,7,194,2022-1,42.26,30.53,11.73,29.597463,1.427825,0.396318,0,0,0,0,0.066667,31.939524
4,2022-01-05,a624fb9a,SE-064c0cec-1189,3099,7,195,2022-1,38.49,42.26,-3.77,29.550181,1.302530,-0.127580,0,0,0,0,-3.700000,26.498206


In [11]:
# Save the reordered DataFrame to a CSV file
data.to_csv('../Data/MergedData/HeatApproachYieldData.csv', index=False)

### Variables Explanation for `HeatApproachYieldData.csv`

1. **Date**:
   - Description: The date when the milk yield was recorded.
   - Datatype: `datetime`
   - Format: `YYYY-MM-DD`
   - Example: `2022-01-01`

2. **FarmName_Pseudo**:
   - Description: A pseudo-identifier for the farm where the data was collected.
   - Datatype: `str`
   - Example: `a624fb9a`

3. **SE_Number**:
   - Description: A unique identifier for the cow, which has been formatted to include the farm and the animal number.
   - Datatype: `str`
   - Example: `SE-064c0cec-1189`

4. **Age**:
   - Description: The age of the cow in days.
   - Datatype: `Int64`
   - Example: `3095`

5. **LactationNumber**:
   - Description: The number assigned to the cow's lactation cycle.
   - Datatype: `Int64`
   - Example: `7`

6. **DaysInMilk**:
   - Description: The number of days the cow has been in milk (lactating) at the time of recording.
   - Datatype: `Int64`
   - Example: `191`

7. **YearSeason**:
   - Description: The seasonal period based on the year and the month range.
   - Datatype: `str`
   - Example: `2022-1`
   - YearSeason parameters in yield datasets:
     - 1: Dec-Feb
     - 2: Mar-May
     - 3: Jun-Aug
     - 4: Sep-Nov

8. **DailyYield**:
   - Description: The total amount of milk produced by the cow in a single day.
   - Datatype: `float`
   - Example: `30.77`

9. **PreviousDailyYield**:
   - Description: The total amount of milk produced by the cow on the previous day.
   - Datatype: `float`
   - Example: `0.0`

10. **DailyYieldChange**:
    - Description: The change in daily milk yield from the previous day.
    - Datatype: `float`
    - Example: `0.0`

11. **ExpectedYield**:
    - Description: The expected amount of milk yield based on certain models or predictions.
    - Datatype: `float`
    - Example: `29.73937171388362`

12. **NormalizedDailyYield**:
    - Description: The daily yield normalized to account for various factors.
    - Datatype: `float`
    - Example: `1.0346553483386214`

13. **NormalizedDailyYieldChange**:
    - Description: The change in normalized daily yield from the previous day.
    - Datatype: `float`
    - Example: `0.0`

14. **HeatStress**:
    - Description: A binary variable indicating the presence of heat stress on the cow.
    - Datatype: `Int64`
    - Example: `0`

15. **Temp15Threshold**:
    - Description: A binary variable indicating if the temperature exceeded 15 degrees Celsius on the given day.
    - Datatype: `Int64`
    - Example: `0`

16. **HW**:
    - Description: A binary variable indicating the presence of a heatwave on the day.
    - Datatype: `Int64`
    - Example: `0`

17. **cum_HW**:
    - Description: Cumulative number of heatwave days up to the current date.
    - Datatype: `Int64`
    - Example: `0`

18. **MeanTemperature**:
    - Description: The mean temperature recorded on the day.
    - Datatype: `float`
    - Example: `-3.025`

19. **MeanTHI_adj**:
    - Description: The mean adjusted Temperature-Humidity Index for the day.
    - Datatype: `float`
    - Example: `28.012944166666667`